In [1]:
import os
import time
import math

def read_dataset(file_path):
    points = []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.split()
            points.append({
                'id': int(data[0]),
                'x': float(data[1]),
                'y': float(data[2])
            })
    return points

def calculate_distance(p1, p2):
    return math.sqrt((p1['x'] - p2['x']) ** 2 + (p1['y'] - p2['y']) ** 2)

def sequential_scan_nn(queries, dataset):
    results = []
    start_time = time.time()
    for query in queries:
        min_distance = float('inf')
        nearest_neighbor = None
        for point in dataset:
            distance = calculate_distance(query, point)
            if distance < min_distance:
                min_distance = distance
                nearest_neighbor = point
        results.append((query['id'], nearest_neighbor))
    total_time = time.time() - start_time
    return results, total_time

if __name__ == '__main__':
    dataset_file = 'restaurant_dataset.txt'
    queries_file = 'query_points.txt'
    output_file = 'sequential_scan_output.txt'

    dataset = read_dataset(dataset_file)
    queries = read_dataset(queries_file)

    results, total_time = sequential_scan_nn(queries, dataset)

    with open(output_file, 'w') as file:
        for query_id, neighbor in results:
            file.write(f"id={query_id}, nearest_id={neighbor['id']}, x={neighbor['x']}, y={neighbor['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")
        file.write(f"Average Time per Query: {total_time / len(queries):.4f} seconds\n")


In [2]:
import heapq

class RTreeNode:
    def __init__(self, points):
        self.points = points
        self.children = []
        self.mbr = self.calculate_mbr(points)
        self.is_leaf = True

    def calculate_mbr(self, points):
        min_x = min(point['x'] for point in points)
        min_y = min(point['y'] for point in points)
        max_x = max(point['x'] for point in points)
        max_y = max(point['y'] for point in points)
        return (min_x, min_y, max_x, max_y)

    def insert(self, point):
        if self.is_leaf:
            self.points.append(point)
            self.mbr = self.calculate_mbr(self.points)
            if len(self.points) > 4:
                self.split()
        else:
            best_child = min(self.children, key=lambda child: self.mbr_enlargement(child.mbr, point))
            best_child.insert(point)
            self.mbr = self.calculate_mbr(self.points)

    def split(self):
        self.is_leaf = False
        self.children = [RTreeNode(self.points[:2]), RTreeNode(self.points[2:])]
        self.points = []
        self.mbr = self.calculate_mbr(self.points)

    def mbr_enlargement(self, mbr, point):
        min_x, min_y, max_x, max_y = mbr
        min_x = min(min_x, point['x'])
        min_y = min(min_y, point['y'])
        max_x = max(max_x, point['x'])
        max_y = max(max_y, point['y'])
        return (max_x - min_x) * (max_y - min_y)

class RTree:
    def __init__(self):
        self.root = RTreeNode([])

    def insert(self, point):
        self.root.insert(point)

def bf_algorithm_nn(queries, rtree):
    def point_distance(p1, p2):
        return math.sqrt((p1['x'] - p2['x']) ** 2 + (p1['y'] - p2['y']) ** 2)

    def mbr_distance(mbr, point):
        min_x, min_y, max_x, max_y = mbr
        dx = max(min_x - point['x'], 0, point['x'] - max_x)
        dy = max(min_y - point['y'], 0, point['y'] - max_y)
        return math.sqrt(dx * dx + dy * dy)

    results = []
    start_time = time.time()
    for query in queries:
        pq = []
        heapq.heappush(pq, (0, rtree.root))
        nearest_neighbor = None
        min_distance = float('inf')

        while pq:
            distance, node = heapq.heappop(pq)
            if node.is_leaf:
                for point in node.points:
                    dist = point_distance(query, point)
                    if dist < min_distance:
                        min_distance = dist
                        nearest_neighbor = point
            else:
                for child in node.children:
                    heapq.heappush(pq, (mbr_distance(child.mbr, query), child))

        results.append((query['id'], nearest_neighbor))
    total_time = time.time() - start_time
    return results, total_time

if __name__ == '__main__':
    dataset_file = 'restaurant_dataset.txt'
    queries_file = 'query_points.txt'
    output_file = 'bf_algorithm_output.txt'

    dataset = read_dataset(dataset_file)
    queries = read_dataset(queries_file)

    rtree = RTree()
    for point in dataset:
        rtree.insert(point)

    results, total_time = bf_algorithm_nn(queries, rtree)

    with open(output_file, 'w') as file:
        for query_id, neighbor in results:
            file.write(f"id={query_id}, nearest_id={neighbor['id']}, x={neighbor['x']}, y={neighbor['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")
        file.write(f"Average Time per Query: {total_time / len(queries):.4f} seconds\n")


ValueError: min() arg is an empty sequence

In [3]:
def divide_dataset(dataset, axis=0):
    dataset.sort(key=lambda point: point['x'] if axis == 0 else point['y'])
    mid = len(dataset) // 2
    return dataset[:mid], dataset[mid:]

def dc_bf_algorithm_nn(queries, dataset):
    left_dataset, right_dataset = divide_dataset(dataset, axis=0)
    
    left_rtree = RTree()
    for point in left_dataset:
        left_rtree.insert(point)

    right_rtree = RTree()
    for point in right_dataset:
        right_rtree.insert(point)

    def dc_nn(query, left_rtree, right_rtree):
        left_result, _ = bf_algorithm_nn([query], left_rtree)
        right_result, _ = bf_algorithm_nn([query], right_rtree)
        
        left_distance = calculate_distance(query, left_result[0][1])
        right_distance = calculate_distance(query, right_result[0][1])
        
        if left_distance < right_distance:
            return left_result[0][1]
        else:
            return right_result[0][1]

    results = []
    start_time = time.time()
    for query in queries:
        nearest_neighbor = dc_nn(query, left_rtree, right_rtree)
        results.append((query['id'], nearest_neighbor))
    total_time = time.time() - start_time
    return results, total_time

if __name__ == '__main__':
    dataset_file = 'restaurant_dataset.txt'
    queries_file = 'query_points.txt'
    output_file = 'output/dc_bf_algorithm_output.txt'

    dataset = read_dataset(dataset_file)
    queries = read_dataset(queries_file)

    results, total_time = dc_bf_algorithm_nn(queries, dataset)

    with open(output_file, 'w') as file:
        for query_id, neighbor in results:
            file.write(f"id={query_id}, nearest_id={neighbor['id']}, x={neighbor['x']}, y={neighbor['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")
        file.write(f"Average Time per Query: {total_time / len(queries):.4f} seconds\n")


ValueError: min() arg is an empty sequence